# Mapas
                

In [1]:
import geocoder 
from geopy import distance
import numpy as np
import folium
from folium import features
import pandas as pd

In [2]:
df= pd.read_excel("r1.xls")
df

,CLIENTE,RUT,NOMBRE,CORRESPONDE,lat,lon,demanda
0,DC,51651,Panaderia heraldo,direccion,-29.962611,-71.260331,0.0
1,PANDERETA,11347208-1,YASNA DEL ROSARIO SALAZAR TORREJON,RENE SCHNEIDER 1878 BOSQ. SN CARLOS,-29.975643,-71.257678,8.0
2,PANATA,7340130-5,HECTOR ALFONSO AROSTEGUI GALLARDO,TALCA 1275 TIERRAS BLANCAS,-29.961857,-71.251173,8.0
3,GIMENA,10231153-1,GIMENA VILLANUEVA HIDALGO,PJE TEKO GUERRA 972 PORTAL SAN RAMON,-29.968230,-71.248536,14.0
4,COLCHAGUA,5180037-0,CLARA POZO,COLCHAGUA 1064 TIERRAS BLANCAS,-29.960446,-71.253913,14.0
5,TARAPACA II,9546989-2,MARIA GALVEZ ROJAS,TARAPACA 370 TIERRAS BLANCAS,-29.955791,-71.262571,8.0
6,JUANA,8070859-9,JUANA YOLANDA MARTINEZ NUNEZ,VICTOR DOMINGO SILVA 418 PEÃUELAS ORIENTE,-29.953516,-71.262063,14.0
7,MILAGRO,08208406-1,EDUARDO GERMAN UGARTE FIGUEROA,SANTO TOMAS DE AQUINO 1525 Milagro II,-29.947509,-71.245638,8.0
8,CLAVERIA,77059329-8,JOSE CLAVERIA ALVAREZ,PSJE.LOS CEREZOS 4839 EL INDIO,-29.943228,-71.262464,14.0
9,TREBOL,7026174-K,ELBA INES ROBLEDO FLORES,LOS PARRONALES 4912 REINA VICTORIA,-29.942713,-71.265922,8.0


In [3]:
df.dropna(inplace=True)
df.reset_index(inplace=True)
df.drop(columns=['index'], inplace=True)

In [5]:
#df.to_excel('r2.xls')

In [6]:
n= df.CLIENTE.count()  

clientes=[]
nodos=[]
demanda=df['demanda']
Q=df.demanda.sum()/2
for i in range(n):
    if i!=0:
        clientes.append(i)
for i in range(n):
    nodos.append(i)

n=df.CLIENTE.count()-1
#Coordenas de cada nodo
latitud=df['lat']
longitud=df['lon']
len(clientes)

45

In [7]:
arcos = [ (i,j) for i in nodos for j in nodos if i != j]
distancia = {(i,j):distance.distance((latitud[i],longitud[i]),(latitud[j],longitud[j])).km for i,j in arcos if i !=j}

In [8]:
from docplex.mp.model import Model
mdl = Model('CVRP')



In [9]:
x = mdl.binary_var_dict(arcos, name = 'x')
u = mdl.continuous_var_dict(nodos, ub = Q , name='u' )

In [10]:
mdl.minimize(mdl.sum(x[i,j] * distancia[i,j] for i,j in arcos))

In [11]:
mdl.add_constraints(mdl.sum(x[i,j] for i in nodos if i !=j) == 1 for j in clientes )
mdl.add_constraints(mdl.sum(x[i,j] for j in nodos if i !=j ) ==1 for i in clientes)

mdl.add_indicator_constraints(mdl.indicator_constraint(x[i,j] , u[i] + demanda[j] == u[j] ) for i,j in arcos if i!=0 and j !=0)

mdl.add_constraints(u[i] >= demanda[i] for i in clientes)

[docplex.mp.LinearConstraint[](u_1,GE,8.0),
 docplex.mp.LinearConstraint[](u_2,GE,8.0),
 docplex.mp.LinearConstraint[](u_3,GE,14.0),
 docplex.mp.LinearConstraint[](u_4,GE,14.0),
 docplex.mp.LinearConstraint[](u_5,GE,8.0),
 docplex.mp.LinearConstraint[](u_6,GE,14.0),
 docplex.mp.LinearConstraint[](u_7,GE,8.0),
 docplex.mp.LinearConstraint[](u_8,GE,14.0),
 docplex.mp.LinearConstraint[](u_9,GE,8.0),
 docplex.mp.LinearConstraint[](u_10,GE,14.0),
 docplex.mp.LinearConstraint[](u_11,GE,8.0),
 docplex.mp.LinearConstraint[](u_12,GE,14.0),
 docplex.mp.LinearConstraint[](u_13,GE,8.0),
 docplex.mp.LinearConstraint[](u_14,GE,8.0),
 docplex.mp.LinearConstraint[](u_15,GE,8.0),
 docplex.mp.LinearConstraint[](u_16,GE,8.0),
 docplex.mp.LinearConstraint[](u_17,GE,8.0),
 docplex.mp.LinearConstraint[](u_18,GE,8.0),
 docplex.mp.LinearConstraint[](u_19,GE,8.0),
 docplex.mp.LinearConstraint[](u_20,GE,14.0),
 docplex.mp.LinearConstraint[](u_21,GE,8.0),
 docplex.mp.LinearConstraint[](u_22,GE,8.0),
 docplex.mp.

In [12]:
mdl.parameters.timelimit=120
solucion = mdl.solve(log_output = True)

CPXPARAM_TimeLimit                               120
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 212.932261 after 0.00 sec. (0.19 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 45 rows and 1 columns.
MIP Presolve modified 990 coefficients.
Aggregator did 990 substitutions.
Reduced MIP has 1080 rows, 3105 columns, and 7020 nonzeros.
Reduced MIP has 2070 binaries, 0 generals, 0 SOSs, and 1980 indicators.
Presolve time = 0.01 sec. (16.68 ticks)
Probing time = 0.05 sec. (29.68 ticks)
Tried aggregator 1 time.
Reduced MIP has 1080 rows, 3105 columns, and 7020 nonzeros.
Reduced MIP has 2070 binaries, 0 generals, 0 SOSs, and 1980 indicators.
Presolve time = 0.00 sec. (5.32 ticks)
Probing time = 0.00 sec. (3.02 ticks)
Clique table members: 1080.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.00 sec. (2.56 ticks)

        Nodes    

In [13]:
mdl.get_solve_status()

<JobSolveStatus.FEASIBLE_SOLUTION: 1>

In [14]:
solucion.display()

solution for: CVRP
objective: 39.246
x_0_10 = 1
x_0_12 = 1
x_0_31 = 1
x_1_30 = 1
x_2_4 = 1
x_3_45 = 1
x_4_0 = 1
x_5_6 = 1
x_6_16 = 1
x_7_40 = 1
x_8_21 = 1
x_9_20 = 1
x_10_29 = 1
x_11_23 = 1
x_12_13 = 1
x_13_5 = 1
x_14_18 = 1
x_15_38 = 1
x_16_9 = 1
x_17_15 = 1
x_18_19 = 1
x_19_22 = 1
x_20_8 = 1
x_21_11 = 1
x_22_2 = 1
x_23_24 = 1
x_24_42 = 1
x_25_3 = 1
x_26_25 = 1
x_27_33 = 1
x_28_37 = 1
x_29_43 = 1
x_30_39 = 1
x_31_35 = 1
x_32_27 = 1
x_33_1 = 1
x_34_0 = 1
x_35_34 = 1
x_36_32 = 1
x_37_36 = 1
x_38_0 = 1
x_39_44 = 1
x_40_14 = 1
x_41_17 = 1
x_42_7 = 1
x_43_28 = 1
x_44_26 = 1
x_45_41 = 1
u_1 = 116.000
u_2 = 223.000
u_3 = 176.000
u_4 = 237.000
u_5 = 57.000
u_6 = 71.000
u_7 = 169.000
u_8 = 115.000
u_9 = 87.000
u_10 = 14.000
u_11 = 131.000
u_12 = 41.000
u_13 = 49.000
u_14 = 191.000
u_15 = 220.000
u_16 = 79.000
u_17 = 212.000
u_18 = 199.000
u_19 = 207.000
u_20 = 101.000
u_21 = 123.000
u_22 = 215.000
u_23 = 139.000
u_24 = 147.000
u_25 = 162.000
u_26 = 154.000
u_27 = 100.000
u_28 = 50.000
u_29 = 2

In [15]:
arcos_activos = [k for k in arcos if x[k].solution_value>0.9]
arcos_activos

[(0, 10),
 (0, 12),
 (0, 31),
 (1, 30),
 (2, 4),
 (3, 45),
 (4, 0),
 (5, 6),
 (6, 16),
 (7, 40),
 (8, 21),
 (9, 20),
 (10, 29),
 (11, 23),
 (12, 13),
 (13, 5),
 (14, 18),
 (15, 38),
 (16, 9),
 (17, 15),
 (18, 19),
 (19, 22),
 (20, 8),
 (21, 11),
 (22, 2),
 (23, 24),
 (24, 42),
 (25, 3),
 (26, 25),
 (27, 33),
 (28, 37),
 (29, 43),
 (30, 39),
 (31, 35),
 (32, 27),
 (33, 1),
 (34, 0),
 (35, 34),
 (36, 32),
 (37, 36),
 (38, 0),
 (39, 44),
 (40, 14),
 (41, 17),
 (42, 7),
 (43, 28),
 (44, 26),
 (45, 41)]

In [16]:
rutas = []
for i in clientes:
    if x[(0,i)].solution_value > 0.9:
        aux = [0,i]
        while i != 0:
            j = i
            for k in nodos:
                if j != k and x[(j,k)].solution_value > 0.9:
                    aux.append(k)
                    i = k
        rutas.append(aux)
print(rutas)

[[0, 10, 29, 43, 28, 37, 36, 32, 27, 33, 1, 30, 39, 44, 26, 25, 3, 45, 41, 17, 15, 38, 0], [0, 12, 13, 5, 6, 16, 9, 20, 8, 21, 11, 23, 24, 42, 7, 40, 14, 18, 19, 22, 2, 4, 0], [0, 31, 35, 34, 0]]


In [17]:
coord_mapa = []

for a in range(len(rutas)):
    for n in rutas[a]:
        coord_mapa.append([(latitud[n]), (longitud[n]) ])
            
            


In [18]:
coord_mapa

[[-29.9626109445584, -71.2603307667609],
 [-29.95620890850089, -71.27182319551973],
 [-29.96443561286758, -71.30422238744909],
 [-29.97009795501173, -71.30961574696715],
 [-29.970704814977765, -71.31245789986998],
 [-29.973015832928912, -71.3091463586135],
 [-29.972100318857237, -71.30813953162566],
 [-29.97243622557632, -71.30522492977806],
 [-29.974322991870913, -71.30525873357426],
 [-29.974021541194205, -71.2648090861478],
 [-29.97564273907185, -71.25767841628416],
 [-29.98151599805824, -71.254469116284],
 [-29.981846026074734, -71.25068380278994],
 [-29.977208426879358, -71.22006193162548],
 [-29.97673944163661, -71.21922411443663],
 [-29.975401191506556, -71.21982922058861],
 [-29.968229999328535, -71.2485357469672],
 [-29.97209815323697, -71.25270818727196],
 [-29.970487834082054, -71.25849151628428],
 [-29.96739090320339, -71.25656760279033],
 [-29.966634702584837, -71.25445772309563],
 [-29.965792367688408, -71.25660328744911],
 [-29.9626109445584, -71.2603307667609],
 [-29.96

In [19]:
mapa = folium.Map(location = [coord_mapa[0][0], coord_mapa[0][1]], zoom_start=12)

colors=['blue','red','green']

for i in range(len(coord_mapa)-1):
    linea=folium.PolyLine(locations =[[coord_mapa[i][0], coord_mapa[i][1]] ,
                                      [coord_mapa[i+1][0], coord_mapa[i+1][1]]] , weight =4, color='red')
    
    mapa.add_child(linea)
    
    

In [20]:
#for i in range(len(latitud)):
#        folium.Marker(
#        [latitud[i], longitud[i]],icon=folium.Icon(color="green")
#    ).add_to(mapa)

In [21]:
mapa